In [2]:
from os.path import join
import glob
import sys
import tqdm
import random
import numpy as np
import re
import numpy as np # linear algebra
import os
import shutil
from pathlib import Path
from tqdm import tqdm
from subprocess import check_output
from shutil import copyfile
import cv2
import matplotlib.pyplot as plt
import pandas as pd
from skimage.morphology import watershed, remove_small_holes, remove_small_objects, label, erosion
from PIL import Image

In [3]:
IMG_WIDTH = 1600
IMG_HEIGHT = 1200
IMG_CHANNELS = 3

#Read images from Original path (the folder given by Luppi)
tot_num = len(os.listdir('DATASET/all_images/Original/'))

In [6]:
#Fix number of test images 58: 2 additional images are insterted manually
#because we need them in the test (see next jupyter cell)

NumberTest = 58
test_names = []
UpperLimit = tot_num

random.seed(99) 
# collect random number to select test set
#untill 58 names are completed

while len(test_names) < NumberTest:
    x = random.randint(0, UpperLimit)
    if x not in test_names:
        test_names.append(x)
    else:
        continue

In [7]:
test_names.append(254) #maccheroni images needed in the test
test_names.append(79) #yellow strip artifact
test_names.sort()

In [8]:
test_names

[1,
 10,
 20,
 21,
 22,
 34,
 38,
 45,
 51,
 52,
 54,
 55,
 58,
 63,
 64,
 68,
 79,
 80,
 86,
 92,
 95,
 97,
 98,
 99,
 103,
 107,
 108,
 118,
 120,
 125,
 132,
 135,
 136,
 137,
 142,
 145,
 150,
 153,
 156,
 159,
 169,
 173,
 174,
 175,
 179,
 183,
 184,
 186,
 194,
 198,
 201,
 202,
 210,
 217,
 220,
 230,
 238,
 240,
 242,
 254]

In [9]:
# Local path >>> swith to absolute path for general code
# Local path >>> swith to absolute path for general code
# Local path >>> swith to absolute path for general code
PATH = 'DATASET/all_images/images/'
PATH_Y = 'DATASET/all_masks/masks/'

PATH_TEST = 'DATASET/TEST/images/'
PATH_TEST_Y = 'DATASET/TEST/masks/'

PATH_TRAIN_VAL = 'DATASET/TRAIN_VAL/all_images/images/'
PATH_TRAIN_VAL_Y= 'DATASET/TRAIN_VAL/all_masks/masks/'

# oure images name are 0.tiff, 1.tiff and so on
# the format is .tiff and no more .TIF as the firts experiment run
# I failed to save in .TIF format when reading from original dataset
# If we want to fix this, we need to change 010_load_file_join_all_images
images_name = os.listdir(PATH)
#select only the number
images_name = [int(x.split('.')[0]) for x in images_name]
#sort the images
#??? Why we sort? I don't remember
images_name.sort()
#restore the original name
images_name = [str(x)+'.tiff' for x in images_name]

In [10]:
# Split images it train_val and test folder
# during the reading the images if the binary mask has more than two values:
# 0 for black and 255 for white this could be related to spurious saving effect from
# GIMP and imageJ software (interpolation???). We need to reset only 0 and 255 value
#We did in the #############Processing############### section below
for ix, im_name in enumerate(images_name):
    
        print(im_name)
        img_x = cv2.imread(str(PATH) + im_name) 
        img_x = cv2.cvtColor(img_x, cv2.COLOR_BGR2RGB)
        img_y = cv2.imread(str(PATH_Y) + im_name)         
        img_y = cv2.cvtColor(img_y, cv2.COLOR_BGR2RGB)[:,:,0:1]
        
        #############Processing###############        
        if len(np.unique(img_y))>2:
            print(' restoring {}'.format(im_name))
        
            ret,img_y = cv2.threshold(img_y,75,255,cv2.THRESH_BINARY)
        
        img_y = img_y.astype(bool)
        img_y = remove_small_objects(img_y,min_size = 15)        
        img_y = img_y.astype(np.uint8)*255
        #############Processing###############   
        
        if int(im_name.split('.')[0]) in test_names:
            print('test')
            img_dir = PATH_TEST+'{}'.format(im_name)
            mask_dir = PATH_TEST_Y+'{}'.format(im_name)
            plt.imsave(fname=img_dir, arr = np.squeeze(img_x))
            plt.imsave(fname=mask_dir,arr = np.squeeze(img_y), cmap='gray')
            
        else:
            
            img_dir = PATH_TRAIN_VAL+'{}'.format(im_name)
            mask_dir = PATH_TRAIN_VAL_Y+'{}'.format(im_name)
            plt.imsave(fname=img_dir, arr = np.squeeze(img_x))
            plt.imsave(fname=mask_dir,arr = np.squeeze(img_y), cmap='gray')

0.tiff
1.tiff
test
2.tiff
3.tiff
4.tiff
5.tiff
6.tiff
7.tiff
8.tiff
9.tiff
10.tiff
test
11.tiff
12.tiff
13.tiff
14.tiff
15.tiff
16.tiff
17.tiff
18.tiff
19.tiff
20.tiff
test
21.tiff
test
22.tiff
test
23.tiff
24.tiff
25.tiff
26.tiff
27.tiff
28.tiff
29.tiff
30.tiff
31.tiff
32.tiff
33.tiff
34.tiff
test
35.tiff
36.tiff
37.tiff
38.tiff
test
39.tiff
40.tiff
41.tiff
42.tiff
43.tiff
44.tiff
45.tiff
test
46.tiff
47.tiff
48.tiff
49.tiff
50.tiff
51.tiff
test
52.tiff
test
53.tiff
54.tiff
test
55.tiff
test
56.tiff
57.tiff
58.tiff
test
59.tiff
60.tiff
61.tiff
62.tiff
63.tiff
test
64.tiff
test
65.tiff
66.tiff
67.tiff
68.tiff
test
69.tiff
70.tiff
71.tiff
72.tiff
73.tiff
74.tiff
75.tiff
76.tiff
77.tiff
78.tiff
79.tiff
test
80.tiff
test
81.tiff
82.tiff
83.tiff
84.tiff
85.tiff
86.tiff
test
87.tiff
88.tiff
89.tiff
90.tiff
91.tiff
92.tiff
test
93.tiff
94.tiff
95.tiff
test
96.tiff
97.tiff
test
98.tiff
test
99.tiff
test
100.tiff
101.tiff
102.tiff
103.tiff
test
104.tiff
105.tiff
106.tiff
107.tiff
test
108.tiff